# Week 2 - Assignment

## Corpus Preprocessing

In [74]:
# Uncomment this line for installing nltk
!pip install nltk

import time
import os
import re
import nltk
from nltk.corpus import stopwords

# Uncomment this line for nltk recognizing stop words
nltk.download('stopwords')

def _preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and special caracters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Divide text into words
    words = text.split()
    
    # Remove portuguese stop words
    stop_words = set(stopwords.words('portuguese'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # Join words separeted by empty space
    processed_text = ' '.join(filtered_words)
    
    return processed_text

def preprocess(input_file_path, output_file_path):
    """
    Preprocess an input file based on constraints

    Constraints are
        1. Punctuation and special caracters must be removed
        2. Portuguese stop words (mas, se, e, a, em) must be removed
        3. Words must be unique (no repetition) and lowercase
    """
    # Open file
    with open(input_file_path, "r", encoding="utf-8") as input_file:
        # Read line and process text
        input_text = input_file.read()
        processed_text = _preprocess_text(input_text)
        # Write processed text
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            output_file.write(processed_text)

## AVL Trees' construction

### Helper

#### Node

In [75]:
class Node:
    """
    A class representing a node in a Binary Search Tree (BST).

    Each node contains a value and references to its left and right children.
    """

    def __init__(self, value):
        """
        Initializes a Node with a given value.

        Parameters:
        - value: The value to be stored in the node.

        Both left and right child references are initialized as None.
        """
        self.value = value
        self.left_child = None
        self.right_child = None
        
class AVLNode(Node):
    """
    Represents a node in an AVL (Adelson-Velsky and Landis) tree,
    which is a self-balancing binary search tree.

    Attributes:
        height (int): The height of the subtree rooted at this node,
                      initializes to 1 when the node is created.
        imbalance (int): The imbalance factor of this node, calculated
                         as the difference between the heights of the left
                         and right subtrees. Initializes to 0.

    Inherits from:
        Node: Inherits attributes and methods from the Node class.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value: The value to be stored in this node.
        """
        super().__init__(value)
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        """
        Calculates the height and imbalance factor of this node based
        on the heights of its left and right children.

        This method assumes that the heights of the children nodes (if they exist)
        are up-to-date.
        """
        # Calculate the height of the left child subtree
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        # Calculate the height of the right child subtree
        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        # Update the height of this node
        self.height = 1 + max(left_height, right_height)

        # Calculate and update the imbalance factor for this node
        self.imbalance = left_height - right_height


#### BST

In [76]:
class BST:
    """
    Represents a Binary Search Tree (BST).

    Attributes:
        root (Node or None): The root node of the tree. Initializes to None for an empty tree.
    """

    def __init__(self):
        """Initializes an empty BST."""
        self.root = None

    def add(self, value):
        """
        Inserts a new value into the BST.

        Args:
            value: The value to be added to the tree.

        If the tree is empty, the value becomes the root. Otherwise, the method uses
        a recursive helper function to find the appropriate position to maintain the BST property.
        """
        if self.root is None:
            self.root = Node(value)
        else:
            self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        Recursively finds the correct position and inserts a value into the BST.

        Args:
            current_node (Node): The node to start the search for the insert position from.
            value: The value to be added to the BST.

        The method determines if the new value should be placed to the left or right of
        the current node. If the target position is empty, the value is inserted.
        Otherwise, the function calls itself recursively with the respective child node.
        """
        if value <= current_node.value:
            if current_node.left_child is None:
                current_node.left_child = Node(value)
            else:
                self._add_recursive(current_node.left_child, value)
        else:
            if current_node.right_child is None:
                current_node.right_child = Node(value)
            else:
                self._add_recursive(current_node.right_child, value)

    def _contains(self, current_node, value):
        """
        Recursively checks if the BST contains the specified value starting from a given node.

        Args:
            current_node (Node): The node to start the search from.
            value: The value to search for in the BST.

        Returns:
            bool: True if the value exists in the subtree rooted at current_node, otherwise False.
        """
        if current_node is None:
            return False
        if current_node.value == value:
            return True
        if value < current_node.value:
            return self._contains(current_node.left_child, value)
        return self._contains(current_node.right_child, value)

    def contains(self, value):
        """
        Checks if the BST contains the specified value.

        Args:
            value: The value to search for in the BST.

        Returns:
            bool: True if the BST contains the value, otherwise False.
        """
        return self._contains(self.root, value)

#### AVL Tree

In [77]:
class AVLTree(BST):
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.
    Inherits all attributes and methods from the BST class and overrides some to maintain the AVL balance property.

    Attributes:
        Inherits all attributes from the BST class.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        super().__init__()

    def add(self, value):
        """
        Overrides the add method in the BST class to handle AVL Tree balancing.
        """
        self.root = self._add_recursive(self.root, value)  # Note that self.root is updated here


    def _add_recursive(self, current_node, value):
        """
        Overrides the BST method to recursively find the correct position and insert a value into the AVL tree.
        This method also ensures the tree remains balanced by updating node heights and performing rotations as needed.

        Args:
            current_node (AVLNode or Node or None): The node from which to start the search for the insert position.
            value (Any): The value to be added to the AVL tree.

        Returns:
            AVLNode: The node that either gets inserted or the node that was already present at that position.

        Notes:
            1. The method first checks if the `current_node` is an instance of the base class `Node`.
              If it is, the method casts it to `AVLNode` to ensure AVL properties are maintained. This is especially
              useful if the first node added to the tree is of type `Node`; this ensures subsequent nodes will be of
              type `AVLNode`.
            2. The method also balances the tree by calling the `_balance` method if the imbalance factor
              of a node reaches 2 or -2 after an insert operation.
        """

        # If the current node is None, return a new AVLNode containing the value
        if current_node is None:
            return AVLNode(value)

        # Check if current_node is of the base class Node and cast it to AVLNode if necessary
        # This is necessary to not change the add() in the BST class.
        # When the first node is added, the type of the root is Node, so we need to cast it
        if isinstance(current_node, Node) and not isinstance(current_node, AVLNode):
            current_node = AVLNode(current_node.value)
            current_node.left_child = self.root.left_child
            current_node.right_child = self.root.right_child
            self.root = current_node

        # Determine whether the value should be inserted to the left or right subtree
        if value <= current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        # Update the height and imbalance factor for the current node
        current_node.calculate_height_and_imbalance()

        # Check if tree balancing is needed and balance if necessary
        if abs(current_node.imbalance) == 2:
            return self._balance(current_node)

        return current_node

    def get_height(self):
        """
        Retrieves the height of the AVL Tree.

        Returns:
            int: The height of the tree rooted at self.root. Returns 0 if the tree is empty.
        """
        if self.root is None:
            return 0
        return self.root.height

    def _rotate_left(self, node):
        """
        Performs a left rotation on the given node and adjusts the height and imbalance attributes.

        A left rotation is used to balance an AVL Tree when the right subtree of a node
        becomes higher than the left subtree. The method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the right subtree of 'node')
        pivot = node.right_child

        # Update the right child of 'node' to be the left child of the pivot
        node.right_child = pivot.left_child

        # Set the left child of the pivot to be the node
        pivot.left_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot


    def _rotate_right(self, node):
        """
        Performs a right rotation on the given node and adjusts the height and imbalance attributes.

        A right rotation is used to balance an AVL Tree when the left subtree of a node
        becomes higher than the right subtree. This method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node around which the rotation will be performed.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the left subtree of 'node')
        pivot = node.left_child

        # Update the left child of 'node' to be the right child of the pivot
        node.left_child = pivot.right_child

        # Set the right child of the pivot to be the node
        pivot.right_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot

    def _balance(self, node):
      """
      Balances the subtree rooted at the given node by performing rotations as needed.

      If the imbalance factor of the given node is 2 or -2, rotations are performed
      to bring the subtree back into balance. This method also takes into account
      the imbalance factors of the child nodes to decide which type of rotation is needed
      (single or double).

      Args:
          node (AVLNode): The root node of the subtree that needs to be balanced.

      Returns:
          AVLNode: The new root node of the balanced subtree.

      Note:
          This method assumes that the height and imbalance factor of each node are up-to-date.
      """

      # Case 1: Left subtree is higher than right subtree
      if node.imbalance == 2:
          pivot = node.left_child
          # Single right rotation
          if pivot.imbalance == 1:
              return self._rotate_right(node)
          # Double rotation: Left-Right
          else:
              node.left_child = self._rotate_left(pivot)
              return self._rotate_right(node)
      # Case 2: Right subtree is higher than left subtree
      else:
          pivot = node.right_child
          # Single left rotation
          if pivot.imbalance == -1:
              return self._rotate_left(node)
          # Double rotation: Right-Left
          else:
              node.right_child = self._rotate_right(pivot)
              return self._rotate_left(node)

### Filling AVL Tree and Array list

In [78]:
def build_structures(output_file_path):
    """
    Insert words present on the preprocessed file into two data structures: AVL and Array List
    """
    # Init data structures
    array_list = []
    avl_tree = AVLTree()
    # Open file with all words
    with open(output_file_path, "r", encoding="utf-8") as file:
        words = file.read().split()
        # Put words into a set because they need to be unique
        unique_words = set(words)
        # Add each word on an AVL and an array list
        for word in unique_words:
            avl_tree.add(word)
            array_list.append(word)
    return avl_tree, array_list

## Automplete function

### AVL

In [79]:
def _get_occurencies_recursive(node, prefix, occurencies):
    """
    Recursive helper function for getting occurencies of a prefix
    
    Args:
        node (AVLNode)  : Current node
        prefix (string) : Prefix to look for
        occurencies (Array) : Array of words that start with prefix
        
    There are four cases
        1. If node is none, we break the recursion because we reach a leaf
        2. If node value starts with prefix, we append into occurencies array and continue searching
        3. If node value is greater than prefix, we descend to the left child
        4. If node value is less than or equal to prefix, the descendo to the right child

    Note: 
        Case 2 is special because we may have a parent whose left and right child have values that starts 
        with the prefix. For example: "n" (prefix) is less than "none", but the right child 
        of "none" might be "null". Therefore, we must traverse "none"'s left and right child
    """
    if node is None:
        return
    if node.value.startswith(prefix):
        occurencies.append(node.value)
        _get_occurencies_recursive(node.left_child, prefix, occurencies)
        _get_occurencies_recursive(node.right_child, prefix, occurencies)
    elif prefix < node.value:
        _get_occurencies_recursive(node.left_child, prefix, occurencies)
    else:
        _get_occurencies_recursive(node.right_child, prefix, occurencies)
    
def get_occurencies_in_avl(avl_tree, prefix):
    """
    Get all occurencies of prefix amongst nodes of an AVL tree

    Args:
        avl_tree (instance of AVL Tree)
        prefix (string)
    """
    occurencies = []
    _get_occurencies_recursive(avl_tree.root, prefix, occurencies)
    return occurencies

### Array List

In [80]:
def get_occurencies_in_list(word_list, prefix):
    """
    Get all occurencies of prefix amongst elements of an Array List
    """
    occurencies = []
    for word in word_list:
        if word.startswith(prefix):
            occurencies.append(word)
    return occurencies

## Gathering everything

In [81]:
def autocomplete(input_file_path, prefix):
    """
    Look for words on a file that start with the prefix

    Args:
        input_file_path (string) : Relative path to the input file
        prefix (string) : Prefix to look for

    Returns:
        object:
            file-length (int) : input_file_path size in KB
            no-words : Amount of unique words on input_file_path
            occurencies : Array of words that start with the prefix
            avl : Time (seconds) an AVL took to search for occurencies
            array-list : Time (seconds) an Array List took to search for occurencies
    """
    # Output filename with sufix "processed"
    file_name, file_extension = os.path.splitext(input_file_path)
    output_file_path = f'{file_name}-processed{file_extension}'

    # Preprocessing
    preprocess(input_file_path, output_file_path)

    # Build AVL and list
    words_avl, words_array_list = build_structures(output_file_path)

    # AVL Perfomance
    start = time.time()
    get_occurencies_in_avl(words_avl, prefix)
    end = time.time()
    avl_time = end - start
    
    # List Perfomance
    start = time.time()
    occurencies = get_occurencies_in_list(words_array_list, prefix)
    end = time.time()
    list_time = end - start

    return {
        'file-length': os.path.getsize(output_file_path)/1024, 
        'no-words': len(words_array_list),
        'occurencies': occurencies, 
        'avl': avl_time, 
        'array-list': list_time
    }

## Testing autocomplete

The JSON files used on this section can be downloaded at [Dados abertos (UFPR)](http://dadosabertos.c3sl.ufpr.br/acordaos/json/). More specifically, the follow files must be donwloaded

- AcordaosRelatorios.json
- AcordaosVotos.json

and put them on `assets` folder

In [84]:
autocomplete('assets/AcordaosRelatorios.json','a')

{'file-length': 114175.9248046875,
 'no-words': 230455,
 'occurencies': ['assu',
  'ano4',
  'assina',
  'arrasto',
  'aceitálos',
  'art17',
  'apoiouse',
  'analisarei',
  'ambrogi',
  'armações',
  'asafe',
  'aiacyda',
  'argila',
  'adaptativo',
  'ancora',
  'adscreve',
  'adaptam',
  'alinhar',
  'anuar',
  'arts109',
  'adpf95df',
  'agex',
  'ausentarse',
  'aafc',
  'amarraram',
  'abstémse',
  'ambulatorial',
  'adiamento',
  'abusivoedoc',
  'apelado',
  'arborização',
  'andap',
  'arquitetos',
  'astorga',
  'autuese',
  'adversa',
  'adequadafl',
  'auditor',
  'arft',
  'acuy',
  'afastaremse',
  'anotada',
  'agravanterecorrente',
  'acesita',
  'aegyptis',
  'aposentados',
  'abcr',
  'apreciarei',
  'adentrarem',
  'anexostjmg',
  'anulála',
  'anista',
  'abundantia',
  'acompanhará',
  'autorapreliminar',
  'agitar',
  'absurdo',
  'arts37',
  'atoreclamado',
  'adoções',
  'abnt',
  'aramis',
  'ascenções',
  'atinente',
  'adotaremos',
  'afeitas',
  'adequou',
 

In [86]:
autocomplete('assets/AcordaosVotos.json','le')

{'file-length': 263193.9658203125,
 'no-words': 349865,
 'occurencies': ['leque',
  'legislador',
  'legislación',
  'leifl',
  'lewandowsi',
  'lecionem',
  'leopoldinense',
  'levinson',
  'lein',
  'leituras',
  'lençóis',
  'leponex',
  'legitimate',
  'leed',
  'leviandade',
  'levavao',
  'levíssima',
  'legalização',
  'leu',
  'leopoldina',
  'levarseá',
  'legendas7',
  'levanos',
  'legalidades',
  'legiferante',
  'lei53',
  'leiamos',
  'lentes',
  'leisrs',
  'legações',
  'legislarão',
  'lembram',
  'lembradas',
  'lex',
  'legitimidades',
  'legislacao',
  'leppaus',
  'lendário',
  'leviana',
  'legitimao',
  'levavam',
  'leonildo',
  'levandolhe',
  'legislativamt',
  'levegrave',
  'legaisfl',
  'lecionam',
  'leyyes',
  'leonar',
  'legatária',
  'leidivan',
  'legitimadaextraordinária',
  'leonildes',
  'legislatividade',
  'lewsandowski',
  'lexclusion',
  'lei1058895',
  'levemente',
  'leonice',
  'levantados',
  'leco',
  'legem',
  'legitimados',
  'legitimid

## Perfomance Analysis

This section is a complement dedicated to compare insertion and searching operations on an AVL and on an Array List

In [83]:
# Building AVL and Array List
avl_tree, array_list = build_structures('assets/AcordaosVotos-processed.json')

# Element to work with
element = "raphael"

# Tree height
print('Tree height')

print('# elements:', len(array_list))
print('AVL height:', avl_tree.get_height())

# Add
print('-'*5)
print('Insertion')

start = time.time()
avl_tree.add(element)
end = time.time()
print('AVL (s):', end - start)

start = time.time()
array_list.append(element)
end = time.time()
print('Array List (s):', end - start)

# Search
print('-'*5)
print('Searching')

start = time.time()
avl_tree.contains(element)
end = time.time()
print('AVL (s):', end - start)

start = time.time()
for el in array_list:
    if el == element:
        break
end = time.time()
print('Array List (s):', end - start)

Tree height
# elements: 349865
AVL height: 22
-----
Insertion
AVL (s): 0.0
Array List (s): 0.0
-----
Searching
AVL (s): 0.0
Array List (s): 0.0707390308380127
